In [1]:
import tensorflow as tf
import numpy as np

In [3]:
def create_geometric_asian_mc_tf_pricer(enable_greeks = True):
    tf.reset_default_graph()
    
    S = tf.placeholder(tf.float32)
    K = tf.placeholder(tf.float32)
    dt = tf.placeholder(tf.float32)
    T = tf.placeholder(tf.float32)
    sigma = tf.placeholder(tf.float32)
    r = tf.placeholder(tf.float32)
    dw = tf.placeholder(tf.float32)
    S_T = S * tf.cumprod(tf.exp((r-sigma**2/2)*dt+sigma*tf.sqrt(dt)*dw), axis=1)
    
    A = tf.pow(tf.reduce_prod(S_T, axis=1), dt/T)
    payout = tf.maximum(A - K, 0)
    npv = tf.exp(-r*T) * tf.reduce_mean(payout)
    
    target_calc = [npv]
    if enable_greeks:
        greeks = tf.gradients(npv, [S, sigma, r, K, T])
        target_calc += [greeks]
    def pricer(S_0, strike, time_to_expiry, implied_vol, riskfree_rate, seed, n_sims, obs):
        if seed != 0:
            np.random.seed(seed)
        stdnorm_random_variates = np.random.randn(n_sims, obs)
        with tf.Session() as sess:
            timedelta = time_to_expiry / obs
            res = sess.run(target_calc, 
                           {
                               S: S_0,
                               K : strike,
                               r : riskfree_rate,
                               sigma: implied_vol,
                               dt : timedelta,
                               T: time_to_expiry,
                               dw : stdnorm_random_variates
                         })
            return res
    return pricer
 
geom_asian_mc_tf_pricer = create_geometric_asian_mc_tf_pricer()
geom_asian_mc_tf_pricer(100, 110, 2, 0.2, 0.03, 1312, 100000, 8)


[4.1968994, [0.3719058, 30.388206, 33.445602, -0.29993924, -89.84526]]

In [88]:
# 1. Modelo de Vasicek: de equilibrio. Proceso de reversión a la media
# 2. Modelo de Jamshidian (Valuación de opciones sobre Bonos)
# en lugar de modelar movimiento browniano sobre subyacente modelamos (movimiento con reversión a la media ó movimiento browniano) sobre tasa de interés
with tf.Session() as sess:
    n_simulations_A = 3
    n_simulations_B = 2
    n_shocks_A = 5
    n_shocks_B = 3
    A = tf.constant([[float(i+1) for _ in range(n_shocks_A)] for i in range(n_simulations_A)])
    print(A.eval())
    
    # Acá es el momento de ejercicio de la opción. Empieza incertidumbre condicional
    B = tf.reshape(A[:,-1], [n_simulations_A,1])
    C1 = tf.tile(B, [n_simulations_B, 1])
    C2 = tf.random.uniform([n_simulations_A * n_simulations_B, n_shocks_B], 0, 1, dtype=tf.float32)
    C = tf.concat([C1,C2], axis=1)
    print(C.eval())
    
# 3. Modelo de Ho & Lee
# 

[[1. 1. 1. 1. 1.]
 [2. 2. 2. 2. 2.]
 [3. 3. 3. 3. 3.]]
[[1.         0.02579439 0.23407316 0.05970359]
 [2.         0.4927708  0.06827819 0.9952179 ]
 [3.         0.59299886 0.31728363 0.5052084 ]
 [1.         0.12036717 0.3536296  0.88829446]
 [2.         0.85759056 0.39999855 0.32045054]
 [3.         0.6603805  0.02178264 0.8374313 ]]


In [70]:
with tf.Session() as sess:
    print(C.shape)

(16, 7)
